---
title: Análisis de Aceites
subtitle: Revisión de completitud
author: Chief Reliability
date: 2023-01-23
---

In [5]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys
from pathlib import Path


import pandas as pd
import polars as pl
import dagster as dg
from kdags.resources import *
from io import BytesIO
import pdfplumber
from datetime import datetime
import re
import os
from kdags.definitions import *
from kdags.readr import Readr
from kdags.resources.ggplot import *

import matplotlib.pyplot as plt
import numpy as np

context = dg.build_asset_context()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
df = Readr.Maintenance.read_oil_analysis(context)

In [2]:
def plot_oil_analysis_time_series(df, equipment_name=None, component_code=None):

    # Ensure sample_date is in datetime format
    df["sample_date"] = pd.to_datetime(df["sample_date"])

    # If no equipment specified, select the first one in the data
    if equipment_name is None:
        equipment_name = df["equipment_name"].unique()[0]

    # Filter data for the selected equipment
    equip_data = df[df["equipment_name"] == equipment_name]

    if equip_data.empty:
        raise ValueError(f"No data found for equipment {equipment_name}")

    # If no component specified, select the one with most samples
    if component_code is None:
        component_counts = equip_data["component_code"].value_counts()
        component_code = component_counts.index[0]

    # Filter data for the selected component
    component_data = equip_data[equip_data["component_code"] == component_code]

    if component_data.empty:
        raise ValueError(
            f"No data found for component {component_code} in equipment {equipment_name}"
        )

    # Sort by date to ensure proper time series
    component_data = component_data.sort_values("sample_date")

    # Select the wear metals to analyze
    wear_metals = ["fe", "cu", "pb", "al"]
    available_metals = [
        metal for metal in wear_metals if metal in component_data.columns
    ]

    # Create the plot data
    plot_data = component_data[["sample_date"] + available_metals].copy()

    # Create a GGPrism plot
    theme = GGPrism()
    fig, ax = theme.create_figure(figsize=(12, 7))

    # Create the line plot with multiple series using GGPrism's line_plot method
    theme.line_plot(
        ax,
        plot_data,
        x="sample_date",
        y=available_metals,
        marker="o",
        markersize=6,
        linewidth=2,
    )

    # Add reference lines for critical thresholds if needed
    # These values would typically come from manufacturer specifications
    thresholds = {
        "fe": 100,  # Example threshold for iron (ppm)
        "cu": 50,  # Example threshold for copper (ppm)
    }

    for metal, threshold in thresholds.items():
        if metal in available_metals:
            ax.axhline(
                y=threshold,
                color=theme.COLORS[available_metals.index(metal)],
                linestyle="--",
                alpha=0.7,
            )

    # Add annotations for significant events or samples
    # For example, highlight the highest Fe reading
    if "fe" in available_metals:
        max_fe_idx = plot_data["fe"].idxmax()
        max_fe_date = plot_data.loc[max_fe_idx, "sample_date"]
        max_fe_value = plot_data.loc[max_fe_idx, "fe"]

        ax.annotate(
            f"Max Fe: {max_fe_value}",
            xy=(max_fe_date, max_fe_value),
            xytext=(10, 10),
            textcoords="offset points",
            arrowprops=dict(arrowstyle="->"),
            fontsize=9,
            color=theme.FIXED_COLORS["axis_color"],
        )

    # Finalize the plot with proper styling using GGPrism's finalize_plot method
    title = f"Wear Metals Trend for {equipment_name} - {component_code}"
    theme.finalize_plot(
        fig,
        ax,
        title=title,
        xlabel="Sample Date",
        ylabel="Concentration (ppm)",
        legend_title="Metal",
    )

    return fig, ax

In [7]:
plot_oil_analysis_time_series(
    df.to_pandas(), equipment_name="EQUIPMENT_NAME", component_code="COMPONENT_CODE"
)

ValueError: No data found for equipment EQUIPMENT_NAME

In [11]:
df.group_by("equipment_name").head(5).write_csv("oil_analysis.csv")

In [3]:
df.filter(pl.col(""))

sample_id,equipment_name,sample_date,received_date,analysis_date,pm_type,equipment_hours,fe,cr,pb,cu,sn,al,ni,ag,si,b,na,mg,ca,ba,p,zn,mo,ti,v,k,oxidation,nitration,sulfation,fuel,water,coolant,soot,viscosity_40c,viscosity_100c,tan,tbn,pq,particles_gt_4,particles_gt_6,particles_gt_14,iso_code,component_code,position_code,is_microfiltered
str,str,date,date,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i8,bool
"""V35574""","""TK872""",2021-07-31,2021-07-31,2021-08-01,"""112D""","""59546""","""11""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""9""","""0""","""0""","""0""","""1""","""0""","""386""","""2""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""710.74""",null,"""0.92""",null,"""8""","""17631""","""2873""","""67""","""21/19/13""","""MT""",2,false
"""V35580""","""TK285""",2021-07-31,2021-07-31,2021-08-01,"""112D""","""3463""","""6""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""11""","""0""","""1""","""1""","""4""","""0""","""319""","""1""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""723.22""",null,"""0.9""",null,"""5""","""1607""","""834""","""40""","""18/17/13""","""MT""",1,true
"""V35576""","""TK872""",2021-07-31,2021-07-31,2021-08-01,"""112D""","""59546""","""3""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""4""","""0""","""0""","""0""","""2""","""0""","""364""","""0""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""723.13""",null,"""0.9""",null,"""5""","""2024""","""518""","""27""","""18/16/12""","""MT""",2,true
"""V35581""","""TK285""",2021-07-31,2021-07-31,2021-08-01,"""112D""","""3463""","""4""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""12""","""0""","""2""","""0""","""4""","""0""","""339""","""0""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""727.28""",null,"""0.9""",null,"""5""","""6025""","""3272""","""44""","""20/19/13""","""MT""",2,true
"""V35573""","""TK872""",2021-07-31,2021-07-31,2021-08-01,"""112D""","""59546""","""7""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""8""","""0""","""0""","""0""","""5""","""0""","""346""","""0""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""721.25""",null,"""0.91""",null,"""6""","""14283""","""2037""","""40""","""21/18/13""","""MT""",1,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""800026394""","""TK286""",2025-03-17,2025-03-18,2025-03-18,"""28D""","""22564""","""8""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""9""","""3""","""0""","""0""","""6""","""0""","""280""","""1""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""724.3""",null,"""0.82""",null,"""14""","""35856.03""","""7265.45""","""650.26""","""22/20/17""","""MT""",2,false
"""800026397""","""TK879""",2025-03-17,2025-03-18,2025-03-18,null,null,"""88""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""19""","""12""","""0""","""0""","""7""","""0""","""312""","""5""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""639.9""",null,"""0.92""",null,"""70""","""215548.02""","""37617.62""","""875.6""","""25/22/17""","""MT""",2,false
"""800026395""","""TK286""",2025-03-17,2025-03-18,2025-03-18,"""28D""","""22564""","""8""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""9""","""3""","""0""","""0""","""7""","""0""","""328""","""3""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""711.5""",null,"""0.82""",null,"""10""","""3352.79""","""1123.79""","""83.49""","""19/17/14""","""MT""",1,true


In [4]:
df

sample_id,equipment_name,sample_date,received_date,analysis_date,pm_type,equipment_hours,fe,cr,pb,cu,sn,al,ni,ag,si,b,na,mg,ca,ba,p,zn,mo,ti,v,k,oxidation,nitration,sulfation,fuel,water,coolant,soot,viscosity_40c,viscosity_100c,tan,tbn,pq,particles_gt_4,particles_gt_6,particles_gt_14,iso_code,component_code,position_code,is_microfiltered
str,str,date,date,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i8,bool
"""V35574""","""TK872""",2021-07-31,2021-07-31,2021-08-01,"""112D""","""59546""","""11""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""9""","""0""","""0""","""0""","""1""","""0""","""386""","""2""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""710.74""",null,"""0.92""",null,"""8""","""17631""","""2873""","""67""","""21/19/13""","""MT""",2,false
"""V35580""","""TK285""",2021-07-31,2021-07-31,2021-08-01,"""112D""","""3463""","""6""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""11""","""0""","""1""","""1""","""4""","""0""","""319""","""1""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""723.22""",null,"""0.9""",null,"""5""","""1607""","""834""","""40""","""18/17/13""","""MT""",1,true
"""V35576""","""TK872""",2021-07-31,2021-07-31,2021-08-01,"""112D""","""59546""","""3""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""4""","""0""","""0""","""0""","""2""","""0""","""364""","""0""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""723.13""",null,"""0.9""",null,"""5""","""2024""","""518""","""27""","""18/16/12""","""MT""",2,true
"""V35581""","""TK285""",2021-07-31,2021-07-31,2021-08-01,"""112D""","""3463""","""4""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""12""","""0""","""2""","""0""","""4""","""0""","""339""","""0""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""727.28""",null,"""0.9""",null,"""5""","""6025""","""3272""","""44""","""20/19/13""","""MT""",2,true
"""V35573""","""TK872""",2021-07-31,2021-07-31,2021-08-01,"""112D""","""59546""","""7""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""8""","""0""","""0""","""0""","""5""","""0""","""346""","""0""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""721.25""",null,"""0.91""",null,"""6""","""14283""","""2037""","""40""","""21/18/13""","""MT""",1,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""800026394""","""TK286""",2025-03-17,2025-03-18,2025-03-18,"""28D""","""22564""","""8""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""9""","""3""","""0""","""0""","""6""","""0""","""280""","""1""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""724.3""",null,"""0.82""",null,"""14""","""35856.03""","""7265.45""","""650.26""","""22/20/17""","""MT""",2,false
"""800026397""","""TK879""",2025-03-17,2025-03-18,2025-03-18,null,null,"""88""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""19""","""12""","""0""","""0""","""7""","""0""","""312""","""5""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""639.9""",null,"""0.92""",null,"""70""","""215548.02""","""37617.62""","""875.6""","""25/22/17""","""MT""",2,false
"""800026395""","""TK286""",2025-03-17,2025-03-18,2025-03-18,"""28D""","""22564""","""8""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""9""","""3""","""0""","""0""","""7""","""0""","""328""","""3""","""0""","""0""","""0""","""0""",null,null,null,null,"""0""",null,null,"""711.5""",null,"""0.82""",null,"""10""","""3352.79""","""1123.79""","""83.49""","""19/17/14""","""MT""",1,true
